In [26]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import torch
from transformers import AlbertModel
import torch.nn as nn

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.albert = AlbertModel.from_pretrained('albert-base-v2')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.albert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.albert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.out(output)

def import_data(model_file):
  n_classes = 2
  model = SentimentClassifier(n_classes)
  model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
  return model

def predict_class(model,review):
  model.eval()  # Put the model in evaluation mode

  # Load the tokenizer
  tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
  text = review
  # Tokenize the example text and create attention masks
  inputs = tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
      truncation=True
  )

  # Get the input IDs and attention mask
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']

  # Predict
  with torch.no_grad():  # No need to compute gradients for predictions
      outputs = model(input_ids, attention_mask=attention_mask)

  # The first element in the outputs is the logits
  logits = outputs[0]

  # Process the outputs
  probabilities = torch.nn.functional.softmax(logits, dim=-1)

  # Get the highest probability class
  predicted_class = torch.argmax(probabilities, dim=-1).numpy()

  # first element of pytorch tensor in float
  probabilities = probabilities.numpy().tolist()
  return (round(max(probabilities)*100, 1), "Positive" if predicted_class == 1 else "Negative")


In [23]:
model = import_data("albert_model.pth")

In [27]:
review = "bad game"
result = predict_class(model,review)

In [28]:
print(result)

(63.7, 'Negative')
